In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.externals import joblib
from collections import Counter
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm

In [2]:
data = joblib.load('data.pkl')

In [3]:
data_dir = 'bbdc_2019_Bewegungsdaten/'

In [4]:
data.keys()

dict_keys(['features', 'labels', 'bounds', 'lens', 'label_encoder'])

In [5]:
def standard_scale(data):
    features = data['features']
    lens = np.concatenate((data['lens']['train'],data['lens']['valid'],data['lens']['test']))
    features = np.concatenate(features, axis=0)
    features = StandardScaler().fit_transform(features)
    ret = []
    l = 0
    for t in lens:
        ret.append(features[l: l + t])
        l += t
    features = np.array(ret)
    data['features'] = features

In [ ]:
standard_scale(data)

In [6]:
def get_split(data, split):
    if split == 'train':
        return data['features'][: data['bounds']['train']]
    elif split == 'valid':
        return data['features'][data['bounds']['train']: data['bounds']['train'] + data['bounds']['valid']]
    elif split == 'test':
        return data['features'][data['bounds']['train'] + data['bounds']['valid']:]

In [7]:
def generate_batch_idx(n, batch_size, randomise=False):
    idx = np.arange(0, n)
    if randomise:
        np.random.shuffle(idx)
    for batch_idx in np.arange(0, n, batch_size):
        yield idx[batch_idx:batch_idx+batch_size]

In [8]:
def generate_batches(data, split, batch_size,
                     randomise=False):
    features = get_split(data, split)
    n = features.shape[0]
    try:
        labels = data['label_encoder'].transform(data['labels'][split])
    except:
        labels = np.zeros(features.shape[0])
    lens = data['lens'][split]
    for batch_idx in generate_batch_idx(n, batch_size, randomise):
        batch_data = features[batch_idx]
        batch_labels = labels[batch_idx]
        batch_lens = lens[batch_idx]
#         batch_data = torch.from_numpy(batch_data).float()
#         batch_labels = torch.from_numpy(labels[batch_idx]).float()
#         lens = torch.from_numpy(lens[batch_idx]).float()
        yield batch_data, batch_labels, batch_lens

In [9]:
# def sort_batch(batch, targets, lengths):
#     """
#     Sort a minibatch by the length of the sequences with the longest sequences first
#     return the sorted batch targes and sequence lengths.
#     This way the output can be used by pack_padded_sequences(...)
#     """
#     perm_idx = np.argsort(lengths)[::-1]
#     seq_lengths = lengths[perm_idx]
#     seq_tensor = batch[perm_idx]
#     target_tensor = targets[perm_idx]
#     return seq_tensor, target_tensor, seq_lengths

def pad_batch(batch, lens):
    max_len = max(lens)
    batch_size = batch.shape[0]
    num_feature = batch[0].shape[1]
    padded_seqs = np.zeros((batch_size, max_len, num_feature))
    
    for i, l in enumerate(lens):
        padded_seqs[i, :l, :] = batch[i][:l]

    return padded_seqs

In [10]:
def torch_batch(batch, targets):
    return torch.from_numpy(batch).float(), torch.from_numpy(targets).long()

In [28]:
def get_preds(model, data, split, batch_size):
    model.eval()
    preds = []
    labels = []
    with torch.no_grad():
        for b_data, b_labels, b_lens in generate_batches(data, split, batch_size, False):
            b_data = pad_batch(b_data, b_lens)
            b_data, b_labels = torch_batch(b_data, b_labels)
            preds.append(model(b_data, b_lens))
            labels.append(b_labels)
    preds = torch.cat(preds, dim=0)
    labels = torch.cat(labels, dim=0)
    return preds, labels

In [36]:
def get_accuracy(preds, labels, le):
    preds = preds.max(dim=1)[1].numpy()
    preds = [le.classes_[i] for i in preds]
    labels = labels.numpy()
    labels = [le.classes_[i] for i in labels]
    return accuracy_score(labels, preds)

In [40]:
class HARNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.rnn = nn.GRU(input_size=14,
                          hidden_size=32,
                          num_layers=1,
                          batch_first=True)
#         self.lin1 = nn.Linear(19, 19, bias=True)
        self.lin2 = nn.Linear(32, 22, bias=True)
    
    def forward(self, data, lens):
        x, _ = self.rnn(data[:, :300, 5:])
        x = torch.cat([seq[-1].view(1, -1) for l, seq in zip(lens, x)], dim=0)
#         x = self.lin1(x)
#         x = F.relu(x)
        x = self.lin2(x)
        return x

In [41]:
num_epochs = 10000
batch_size = 32
objective = nn.CrossEntropyLoss()
model = HARNet()
optimiser = torch.optim.Adam(model.parameters(), weight_decay=0.0)
running_loss = 0
running_batch = 0
min_valid_loss = float('inf')

for epoch in range(1, num_epochs + 1):
    with tqdm(enumerate(generate_batches(data, 'train', batch_size, True), 1)) as pbar:
        model.train()
        for batch_num, (batch_data, batch_labels, batch_lens) in pbar:
            batch_data = pad_batch(batch_data, batch_lens)
            batch_data, batch_labels = torch_batch(batch_data, batch_labels)
            optimiser.zero_grad()
            preds = model(batch_data, batch_lens)
            loss = objective(preds, batch_labels)
            loss.backward()
            optimiser.step()
            running_loss += loss.item()
            running_batch += 1
            pbar.set_description(f'[Epoch: {epoch}] | Batch {batch_num} | Loss: {running_loss/running_batch}')
            
    valid_preds, valid_labels = get_preds(model, data, 'valid', 64)
    valid_loss = objective(valid_preds, valid_labels).item()
    if valid_loss < min_valid_loss:
        print(f'Validation loss improved from {min_valid_loss} to {valid_loss}')
        acc = get_accuracy(valid_preds, valid_labels, data['label_encoder'])
        print(f'Validation accuracy: {acc}')
        min_valid_loss = valid_loss
        with open('best_rnn_model.pt', 'wb') as f:
            torch.save(model.state_dict(), f)
    else:
        print('Validation loss did not improve')

[Epoch: 1] | Batch 165 | Loss: 2.920111407655658: : 165it [00:39,  5.18it/s] 
0it [00:00, ?it/s]

Validation loss improved from inf to 2.7040443420410156
Validation accuracy: 0.17213842058562556


[Epoch: 2] | Batch 165 | Loss: 2.7633545702153985: : 165it [00:33,  4.14it/s]
0it [00:00, ?it/s]

Validation loss improved from 2.7040443420410156 to 2.559023380279541
Validation accuracy: 0.16858917480035493


[Epoch: 3] | Batch 165 | Loss: 2.666747970773716: : 165it [00:34,  5.90it/s] 
[Epoch: 4] | Batch 1 | Loss: 2.666240318167594: : 1it [00:00,  5.22it/s]

...

Validation loss did not improve


[Epoch: 125] | Batch 165 | Loss: 1.3136375039794228: : 165it [00:22,  7.50it/s]
[Epoch: 126] | Batch 1 | Loss: 1.313603371323964: : 1it [00:00,  7.91it/s]

Validation loss did not improve


[Epoch: 126] | Batch 161 | Loss: 1.3100012689757183: : 161it [00:24,  6.70it/s]


KeyboardInterrupt: 

In [20]:
preds.max(dim=1)

(tensor([0.9428, 1.2849, 0.8000, 0.7982, 4.9522, 1.0544, 1.2595, 1.0704, 1.5669,
         0.7547], grad_fn=<MaxBackward0>),
 tensor([21,  0, 20, 20, 12,  0, 20,  6, 20, 17]))

In [21]:
batch_labels

tensor([ 1, 18, 16,  9, 11, 16,  7, 13,  1, 10])

In [23]:
data['label_encoder'].classes_[16]

'stand-to-sit'

In [24]:
data['label_encoder'].classes_[17]

'v-cut-left-Lfirst'

In [33]:
preds.max(dim=1)[1].numpy()

array([21,  9, 17, ...,  8,  0,  4])

In [3]:
np.arange(0, 10, 3)

array([0, 3, 6, 9])